In [1]:
import findspark
findspark.init()

In [57]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import length
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.ml.feature import CountVectorizer, IDF, StringIndexer
from pyspark.ml.classification import NaiveBayes, RandomForestClassifier, LogisticRegression
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.linalg import Vector
from pyspark.ml import Pipeline
from pyspark.sql.functions import lit
from bs4 import BeautifulSoup
from pyspark import keyword_only
from pyspark.ml import Transformer
from pyspark.sql.functions import when
import pyspark.sql.functions as F
from pyspark.mllib.evaluation import MulticlassMetrics, BinaryClassificationMetrics
from datetime import datetime
from pyspark.ml import Transformer
from pyspark.sql.types import FloatType
from pyspark.ml.param.shared import HasInputCol, HasOutputCol 
from pyspark.sql.functions import udf
from pyspark.sql.types import StringType
from pyspark.ml.evaluation import MulticlassClassificationEvaluator, BinaryClassificationEvaluator

In [3]:
spark = SparkSession.builder.appName('Rating').getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/28 02:18:07 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/28 02:18:07 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


### Load Data

In [4]:
df1 = spark.read.csv("shopee_final.csv",header=True
                      ,inferSchema=True)

In [5]:
df1.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- processed_text: string (nullable = true)
 |-- class: string (nullable = true)



In [6]:
#df1 = df1.withColumn('class',lit('fake'))

In [7]:
df2 = df1.withColumn("class", when(df1['class'] == "negative","negative") \
      .when(df1['class'] == "neutral","negative") \
      .otherwise(df1['class']))
df2.show()

23/03/28 02:18:14 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , processed_text, class
 Schema: _c0, processed_text, class
Expected: _c0 but found: 
CSV file: file:///Users/tony.ng/Documents/DS-ML/final_project/project1/shopee_final.csv
+---+--------------------+--------+
|_c0|      processed_text|   class|
+---+--------------------+--------+
|  0|miếng dán hơi_dầy...|negative|
|  1|miếng dán rất_tồi...|negative|
|  2|cường_lực trắng m...|negative|
|  3|hàng cảm_quan đầu...|negative|
|  4|chất_lượng cường_...|negative|
|  5|kính chất_lượng k...|negative|
|  6|kính cường_lực dở...|negative|
|  7|bị_vỡ bắt_làm hoà...|negative|
|  8|kính bụi dính kín...|negative|
|  9|đóng hàng cường_l...|negative|
| 10|hàng cường_lực xi...|negative|
| 11|không_vừa gần_chấ...|negative|
| 12|                  bé|negative|
| 13|         đo thử chán|negative|
| 14|   hàng miếng bảo_vệ|negative|
| 15|sản_phẩm miết rất...|negative|
| 16|chính_xác sản_phẩ...|negative|
| 17|     hàn

In [8]:
df2.count()

934543

In [9]:
df2.groupby('class').count().show()

+--------+------+
|   class| count|
+--------+------+
|positive|636052|
|negative|298491|
+--------+------+



### Precprocess

In [10]:
df2 = df2.withColumn('length', length(df2['processed_text']))

In [11]:
df2.show(5)

23/03/28 02:18:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , processed_text, class
 Schema: _c0, processed_text, class
Expected: _c0 but found: 
CSV file: file:///Users/tony.ng/Documents/DS-ML/final_project/project1/shopee_final.csv
+---+--------------------+--------+------+
|_c0|      processed_text|   class|length|
+---+--------------------+--------+------+
|  0|miếng dán hơi_dầy...|negative|    28|
|  1|miếng dán rất_tồi...|negative|    57|
|  2|cường_lực trắng m...|negative|    23|
|  3|hàng cảm_quan đầu...|negative|    63|
|  4|chất_lượng cường_...|negative|   101|
+---+--------------------+--------+------+
only showing top 5 rows



In [12]:
df2.groupBy('class').mean().show()

23/03/28 02:18:16 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , processed_text, class
 Schema: _c0, processed_text, class
Expected: _c0 but found: 
CSV file: file:///Users/tony.ng/Documents/DS-ML/final_project/project1/shopee_final.csv
+--------+-----------------+------------------+
|   class|         avg(_c0)|       avg(length)|
+--------+-----------------+------------------+
|positive|477209.3752444769|44.500778346973405|
|negative| 446093.398494427| 41.68132393148086|
+--------+-----------------+------------------+



In [13]:
data = df2.select('processed_text','class','length')

In [14]:
data.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|miếng dán hơi_dầy...|negative|    28|
|miếng dán rất_tồi...|negative|    57|
|cường_lực trắng m...|negative|    23|
|hàng cảm_quan đầu...|negative|    63|
|chất_lượng cường_...|negative|   101|
|kính chất_lượng k...|negative|    41|
|kính cường_lực dở...|negative|   126|
|bị_vỡ bắt_làm hoà...|negative|    32|
|kính bụi dính kín...|negative|    33|
|đóng hàng cường_l...|negative|    32|
|hàng cường_lực xi...|negative|    61|
|không_vừa gần_chấ...|negative|    22|
|                  bé|negative|     2|
|         đo thử chán|negative|    11|
|   hàng miếng bảo_vệ|negative|    17|
|sản_phẩm miết rất...|negative|    89|
|chính_xác sản_phẩ...|negative|   151|
|     hàng hãng lưu_ý|negative|    15|
|chất_lượng sản_ph...|negative|   122|
|cường_lực mô_tả b...|negative|    82|
+--------------------+--------+------+
only showing top 20 rows



### Null

In [15]:
null_data = data.filter(data['processed_text'].isNull())

In [16]:
null_data.count()

33669

In [17]:
data = data.filter(data['processed_text'].isNotNull())

### Duplicated

In [18]:
data.dropDuplicates(['processed_text']).show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|aaaaaaa giá rẻ ch...|positive|    32|
|aaaaaaaaaaaaaaaaa...|positive|    53|
|aaaaaaaaaaaaaaaaa...|positive|    49|
|aaaaaaaaaaaaaaaaa...|positive|    56|
|abajajaj djsjana ...|positive|    28|
|acid dịch_vị giúp...|negative|    99|
|add ground liver ...|positive|    21|
|addjficuc êwưabja...|positive|    49|
|     adidas gửi nhầm|negative|    15|
|admin kích_hoạt b...|negative|    27|
|      agcejzvebgrhdg|positive|    14|
|agsgdhbdjs djdjsh...|positive|    58|
|ahslffl shwodlddj...|positive|    42|
|airpod đầu bị_lỗi...|negative|    88|
|akfjso kakdjdnd đ...|positive|    47|
|               akwkd|negative|     5|
|amin đóng gói hàn...|positive|    40|
|android không_khẳ...|negative|    41|
|         android máy|negative|    11|
|aoa thahsjhsbsjxa...|positive|    61|
+--------------------+--------+------+
only showing top 20 rows



In [20]:
data.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|miếng dán hơi_dầy...|negative|    28|
|miếng dán rất_tồi...|negative|    57|
|cường_lực trắng m...|negative|    23|
|hàng cảm_quan đầu...|negative|    63|
|chất_lượng cường_...|negative|   101|
|kính chất_lượng k...|negative|    41|
|kính cường_lực dở...|negative|   126|
|bị_vỡ bắt_làm hoà...|negative|    32|
|kính bụi dính kín...|negative|    33|
|đóng hàng cường_l...|negative|    32|
|hàng cường_lực xi...|negative|    61|
|không_vừa gần_chấ...|negative|    22|
|                  bé|negative|     2|
|         đo thử chán|negative|    11|
|   hàng miếng bảo_vệ|negative|    17|
|sản_phẩm miết rất...|negative|    89|
|chính_xác sản_phẩ...|negative|   151|
|     hàng hãng lưu_ý|negative|    15|
|chất_lượng sản_ph...|negative|   122|
|cường_lực mô_tả b...|negative|    82|
+--------------------+--------+------+
only showing top 20 rows



In [26]:
data.count()

900874

In [27]:
data.groupby('class').count().show()

+--------+------+
|   class| count|
+--------+------+
|positive|614122|
|negative|286752|
+--------+------+



### Resampling

In [30]:
from pyspark.sql.functions import col, explode, array, lit
major_df = data.filter(col("class") == 'positive')
minor_df = data.filter(col("class") == 'negative')
ratio = int(major_df.count()/minor_df.count())
print("ratio: {}".format(ratio))


ratio: 2


In [31]:
sampled_majority_df = major_df.sample(False, 1/ratio)
combined_df_2 = sampled_majority_df.unionAll(minor_df)
combined_df_2.show()

+--------------------+--------+------+
|      processed_text|   class|length|
+--------------------+--------+------+
|dán hơi_lỗi ảnh t...|positive|    25|
|            kèm giấy|positive|     8|
|miếng kiểu thèm b...|positive|    76|
|gói hàng rất_có t...|positive|   123|
|dán mặt lưng xr t...|positive|   109|
|sản_phẩm tốt khôn...|positive|    53|
|không_đủ hàng khô...|positive|    50|
|gãy hẳn nửa giá c...|positive|    33|
|hàng chất_lượng s...|positive|    71|
|ảnh thực dán vô h...|positive|    23|
|kính sọc chống tr...|positive|    37|
|tạm hình quảng_cá...|positive|    31|
|dán dán màn_hình ...|positive|    35|
|                  vỡ|positive|     2|
|          cơ_bản giá|positive|    10|
|hàng remax autobo...|positive|    30|
|cường_lực dán tốt...|positive|    40|
|đóng gói đẹp khs ...|positive|    52|
|tạm ổn ádfghjklzm...|positive|    27|
|hàng đóng_gói kỹ ...|positive|    30|
+--------------------+--------+------+
only showing top 20 rows



In [33]:
combined_df_2.groupby('class').count().show()

+--------+------+
|   class| count|
+--------+------+
|positive|307836|
|negative|286752|
+--------+------+



### Feature & Transform

In [34]:
class BsTextExtractor(Transformer, HasInputCol, HasOutputCol):
    
    @keyword_only
    def __init__(self, inputCol=None, outputCol=None):
        super(BsTextExtractor, self).__init__() 
        kwargs = self._input_kwargs 
        self.setParams(**kwargs)
        
    @keyword_only
    def setParams(self, inputCol=None, outputCol=None): 
        kwargs = self._input_kwargs
        return self._set(**kwargs)
    def _transform(self, dataset):
        def f(s):
            cleaned_text = BeautifulSoup(s).text 
            return cleaned_text
        t = StringType()
        out_col = self.getOutputCol()
        in_col = dataset[self.getInputCol()]
        return dataset.withColumn(out_col, udf(f, t)(in_col))

In [35]:
text_extractor = BsTextExtractor(inputCol="processed_text", outputCol="cleaned_text")
tokenizer = Tokenizer(inputCol='cleaned_text', outputCol='token_text')
stopremove= StopWordsRemover(inputCol='token_text', outputCol='stop_tokens')
count_vec = CountVectorizer(inputCol='stop_tokens', outputCol='c_vec')
idf = IDF(inputCol='c_vec', outputCol='tf_idf')
ham_spam_to_num = StringIndexer(inputCol='class',outputCol='label', handleInvalid='keep')

In [36]:
clean_up = VectorAssembler(inputCols =['tf_idf','length'],
                           outputCol='features')

In [37]:
data_prep_pipe = Pipeline(stages=[ham_spam_to_num,
                                  text_extractor,
                                  tokenizer,
                                  stopremove,
                                  count_vec,
                                  idf,
                                  clean_up])

In [38]:
cleaner = data_prep_pipe.fit(combined_df_2)

In [43]:
clean_data = cleaner.transform(combined_df_2)

In [44]:
clean_data = clean_data.select('label','features')

In [45]:
clean_data.show()

23/03/28 02:25:08 WARN DAGScheduler: Broadcasting large task binary with size 1900.5 KiB
+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(63765,[35,57,69,...|
|  0.0|(63765,[142,191,6...|
|  0.0|(63765,[53,187,65...|
|  0.0|(63765,[1,18,21,5...|
|  0.0|(63765,[13,69,72,...|
|  0.0|(63765,[0,20,69,1...|
|  0.0|(63765,[1,4,28,67...|
|  0.0|(63765,[15,244,42...|
|  0.0|(63765,[0,1,2,27,...|
|  0.0|(63765,[35,69,284...|
|  0.0|(63765,[20,214,22...|
|  0.0|(63765,[22,25,28,...|
|  0.0|(63765,[18,53,69,...|
|  0.0|(63765,[309,63764...|
|  0.0|(63765,[15,1386,6...|
|  0.0|(63765,[1,2,5204,...|
|  0.0|(63765,[1,20,41,6...|
|  0.0|(63765,[1,5,21,24...|
|  0.0|(63765,[22,27,537...|
|  0.0|(63765,[1,7,50,14...|
+-----+--------------------+
only showing top 20 rows



In [46]:
clean_data.groupBy('label').count().show()

+-----+------+
|label| count|
+-----+------+
|  0.0|307836|
|  1.0|286752|
+-----+------+



In [47]:
(train, test) = clean_data.randomSplit([0.7,0.3])

## Buil Model

### Naive Bayes

In [50]:
nb = NaiveBayes()
prediction = nb.fit(train)
test_results = prediction.transform(test)
start_time = datetime.now()
train_time = datetime.now() - start_time  
test_results.show()
print(train_time)

23/03/28 02:26:17 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:26:40 WARN DAGScheduler: Broadcasting large task binary with size 5.2 MiB


23/03/28 02:26:40 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(63765,[0,1,2,4,5...|[-322.26715326891...|[0.99999915565628...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-520.00195122323...|[1.0,3.6284485193...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0,1,2,4,7...|[-161.36232498969...|[1.0,1.4887069201...|       0.0|
|  0.0|(63765,[0

In [51]:
test_results.groupBy('label', 'prediction').count().show()

23/03/28 02:26:50 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB


23/03/28 02:27:12 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       1.0|13963|
|  0.0|       0.0|78292|
|  1.0|       1.0|68842|
|  1.0|       0.0|17618|
+-----+----------+-----+



### Report

In [59]:
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels = test_results.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels = preds_and_labels.select(['prediction','label'])

metrics = MulticlassMetrics(preds_and_labels.rdd.map(tuple))
print(metrics.confusionMatrix().toArray())

23/03/28 02:32:56 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/28 02:33:17 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/28 02:33:40 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


23/03/28 02:33:40 WARN DAGScheduler: Broadcasting large task binary with size 6.2 MiB


[[78292. 13963.]
 [17618. 68842.]]


In [60]:
acc_eva = MulticlassClassificationEvaluator()
acc = acc_eva.evaluate(test_results)
print('Accuracy of model: {}'.format(acc))

23/03/28 02:33:54 WARN DAGScheduler: Broadcasting large task binary with size 6.3 MiB


Accuracy of model: 0.8230968396441363


### Logistic Regression

In [61]:
lg = LogisticRegression(maxIter=10, regParam=0.3)
pre_lg = lg.fit(train)
result_lg = pre_lg.transform(test)
start_time = datetime.now()
train_time = datetime.now() - start_time 
print(train_time)

23/03/28 02:34:22 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:34:43 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:34:44 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:34:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.JNIBLAS
23/03/28 02:34:45 WARN InstanceBuilder$NativeBLAS: Failed to load implementation from:dev.ludovic.netlib.blas.ForeignLinkerBLAS


23/03/28 02:35:05 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB
23/03/28 02:35:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
23/03/28 02:35:06 WARN BLAS: Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


...

23/03/28 02:35:22 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:35:23 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:35:23 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


23/03/28 02:35:24 WARN DAGScheduler: Broadcasting large task binary with size 5.3 MiB


0:00:00.000106


In [62]:
result_lg.groupBy('label', 'prediction').count().show()

23/03/28 02:35:31 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB


23/03/28 02:35:54 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB
+-----+----------+-----+
|label|prediction|count|
+-----+----------+-----+
|  0.0|       1.0|13699|
|  0.0|       0.0|78556|
|  1.0|       1.0|73996|
|  1.0|       0.0|12464|
+-----+----------+-----+



### Report

In [64]:
#important: need to cast to float type, and order by prediction, else it won't work
preds_and_labels_lg = result_lg.select(['prediction','label']).withColumn('label', F.col('label').cast(FloatType())).orderBy('prediction')

#select only prediction and label columns
preds_and_labels_lg = preds_and_labels_lg.select(['prediction','label'])

metrics_lg = MulticlassMetrics(preds_and_labels_lg.rdd.map(tuple))
print(metrics_lg.confusionMatrix().toArray())

23/03/28 02:37:03 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB


23/03/28 02:37:25 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB


23/03/28 02:37:46 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB


23/03/28 02:37:47 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB


[[78556. 13699.]
 [12464. 73996.]]


In [65]:
acc_eva_lg = MulticlassClassificationEvaluator()
acc_lg = acc_eva_lg.evaluate(result_lg)
print('Accuracy of model: {}'.format(acc_lg))

23/03/28 02:37:53 WARN DAGScheduler: Broadcasting large task binary with size 6.7 MiB


Accuracy of model: 0.8536307313470617


#### Nhận xét:
- đối với việc sử dụng Pyspark, thời gian xử lý nhanh hơn, NB nhanh hơn về tốc độ xử lý so với logistic Regression
- Tuy nhiên kết quả thì logistic lại tốt hơn về độ chính xác

##### Như vậy Pyspark cho kết quả tốt hơn cả về thời gian xử lý lân độ chính xác